<a href="https://colab.research.google.com/github/dk-wei/super-duper-transformer/blob/main/bert_classification_(CLS_%2B_Fine_tune_%E4%B8%AD%E6%96%87%E7%89%88%E6%9C%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install transformers==3

In [5]:
#part2: bert feature-base
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as tfs
import warnings

warnings.filterwarnings('ignore')

# Part 1: 直接利用BERT提取`[CLS]`特征的方式进行建模

## 加载数据集

In [6]:
train_df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [7]:
train_set = train_df[:3000]

print("Train set shape:", train_set.shape)

Train set shape: (3000, 2)


In [8]:
train_set[1].value_counts()

1    1565
0    1435
Name: 1, dtype: int64

可以看出，积极和消极的标签基本对半分。

## 利用BERT进行特征抽取

在这里，我们利用BERT对数据集进行特征抽取，即把输入数据经过BERT模型，来获取输入数据的特征，这些特征包含了整个句子的信息，是语境层面的。这种做法类似于EMLo的特征抽取。需要注意的是，这里并没有使用到BERT的微调，因为BERT并不参与后面的训练，仅仅进行特征抽取操作。

In [9]:
model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

我们使用预训练好的"`bert-base-uncased`"模型参数进行处理，采用的模型是 `BertModel` ，采用的分词器是 `BertTokenizer` 。由于我们的输入句子是英文句子，所以需要先分词；然后把单词映射成词汇表的索引，再喂给模型。实际上Bert的分词操作，不是以传统的单词为单位的，而是以 `wordpiece` 为单位，这是比单词更细粒度的单位。我们执行以下代码：

In [10]:
train_tokenized = train_set[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

然后，为了提升训练速度，我们需要把句子都处理成同一个长度，即常见的pad操作，我们在短的句子末尾添加一系列的`[PAD]`符号：

In [11]:
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_padded = np.array([i + [0] * (train_max_len-len(i)) for i in train_tokenized.values])
print("train set shape:",train_padded.shape)

train set shape: (3000, 66)


最后，我们还需要让模型知道，哪些词是不用处理的，即上面我们添加的[PAD]符号, 其实就是attention-mask的功能：



In [12]:
print(train_padded[0])
train_attention_mask = np.where(train_padded != 0, 1, 0)
print(train_attention_mask[0])

[  101  1037 18385  1010  6057  1998  2633 18276  2128 16603  1997  5053
  1998  1996  6841  1998  5687  5469  3152   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
train_input_ids = torch.tensor(train_padded).long()
train_attention_mask = torch.tensor(train_attention_mask).long()
with torch.no_grad():
    train_last_hidden_states = model(train_input_ids, attention_mask=train_attention_mask)

我们来看以下Bert模型给我们的输出是什么样的：

In [14]:
train_last_hidden_states[0].size()

torch.Size([3000, 66, 768])

第一维的是样本数量，第二维的是序列长度，第三维是特征数量。也就是说，Bert对于我们的每一个位置的输入，都会输出一个对应的特征向量。

## 切分数据成训练集和测试集

In [15]:
train_features = train_last_hidden_states[0][:,0,:].numpy()
train_labels = train_set[1]


请注意：我们使用[:,0,:]来提取序列第一个位置的输出向量，因为第一个位置是[CLS]，比起其他位置，该向量应该更具有代表性，蕴含了整个句子的信息。紧接着，我们利用sklearn库的方法来把数据集切分成训练集和测试集。

![](https://camo.githubusercontent.com/009eabd7b7697055256ab3655f048d5c31967e1b/68747470733a2f2f6a616c616d6d61722e6769746875622e696f2f696d616765732f64697374696c424552542f626572742d6f75747075742d74656e736f722d73656c656374696f6e2e706e67)

In [16]:
train_features, test_features, train_labels, test_labels = train_test_split(train_features, train_labels)

## 使用逻辑回归进行训练

在这一部分，我们使用sklearn的逻辑回归模块对我们的训练集进行拟合，最后在测试集上进行评价：

In [17]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lr_clf.score(test_features, test_labels)

0.868

经过逻辑回归模型的拟合，其准确率达到了79.21，分类效果还不错。那么，我们还能进一步提升吗？

# Part 2: 利用BERT基于微调的方式进行建模

在上一部分，我们利用了Bert抽取特征的能力进行建模，直接提取了Bert的输出特征，再输入给一个线性层以预测。但Bert本身的不参与模型的训练。

现在我们采取另一种方式，即fine-tuned，Bert与线性层一起参与训练，反向传播会更新bert和classifier二者的参数，使得Bert模型更加适合这个分类任务。那么，让我们开始吧~

## 建立模型



In [22]:
#part 2 - bert fine-tuned
import torch
from torch import nn
from torch import optim
import transformers as tfs
import math

class BertClassificationModel(nn.Module):
    def __init__(self):
        super(BertClassificationModel, self).__init__()   
        model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-uncased')         
        self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        self.bert = model_class.from_pretrained(pretrained_weights)
        self.dense = nn.Linear(768, 2)  #bert默认的隐藏单元数是768， 输出单元是2，表示二分类
        
    def forward(self, batch_sentences):
        #print(batch_sentences[4])

        #上面那些构造attention mask以及tokenization直接可以用batch_encode_plus一步到位
        batch_tokenized = self.tokenizer.batch_encode_plus(batch_sentences, 
                                                           add_special_tokens=True,
                                                           max_length=66, 
                                                           padding='max_length', # padding有很多种，可以自行查阅documents
                                                           truncation=True,
                                                           #pad_to_max_length=True
                                                           )      #tokenize、add special token、pad
        
        input_ids = torch.tensor(batch_tokenized['input_ids'])
        attention_mask = torch.tensor(batch_tokenized['attention_mask'])
        
        bert_output = self.bert(input_ids, attention_mask=attention_mask)
        
        bert_cls_hidden_state = bert_output[0][:,0,:]       #提取[CLS]对应的隐藏状态, 即使是training，用的也是[CLS]的embedding
        
        linear_output = self.dense(bert_cls_hidden_state)    #卧槽，dense一下就可以直接输出probability了
        return linear_output

模型很简单，关键代码都在上面注释了。其主要构成是在bert模型的[CLS]输出位置接上一个线性层，用以预测句子的分类。

## 数据分批

下面我们对原来的数据集进行一些改造，分成batch_size为64大小的数据集，以便模型进行批量梯度下降。

In [23]:
sentences = train_set[0].values
targets = train_set[1].values
train_inputs, test_inputs, train_targets, test_targets = train_test_split(sentences, targets)

batch_size = 64
batch_count = int(len(train_inputs) / batch_size)
batch_train_inputs, batch_train_targets = [], []

for i in range(batch_count):
    batch_train_inputs.append(train_inputs[i*batch_size : (i+1)*batch_size])
    batch_train_targets.append(train_targets[i*batch_size : (i+1)*batch_size])

## 训练模型



In [24]:
#train the model
epochs = 3
lr = 0.01
print_every_batch = 10

bert_classifier_model = BertClassificationModel()
optimizer = optim.SGD(bert_classifier_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print_avg_loss = 0
    for i in range(batch_count):
        # forward-propagation
        inputs = batch_train_inputs[i]
        labels = torch.tensor(batch_train_targets[i])
        optimizer.zero_grad()

        outputs = bert_classifier_model(inputs)
        loss = criterion(outputs, labels)

        # back-propagation
        loss.backward()
        optimizer.step()
        
        print_avg_loss += loss.item()
        if i % print_every_batch == (print_every_batch-1):
            print("Batch: %d, Loss: %.4f" % ((i+1), print_avg_loss/print_every_batch))
            print_avg_loss = 0


Batch: 10, Loss: 0.6775
Batch: 20, Loss: 0.5447
Batch: 30, Loss: 0.5848
Batch: 10, Loss: 0.4557
Batch: 20, Loss: 0.3258
Batch: 30, Loss: 0.3545
Batch: 10, Loss: 0.3087
Batch: 20, Loss: 0.2010
Batch: 30, Loss: 0.2941


## 模型评价



In [25]:
# eval the trained model
total = len(test_inputs)
hit = 0
with torch.no_grad():
    for i in range(total):
        outputs = bert_classifier_model([test_inputs[i]])
        _, predicted = torch.max(outputs, 1)
        if predicted == test_targets[i]:
            hit += 1

print("Accuracy: %.2f%%" % (hit / total * 100))

Accuracy: 86.53%
